In [1]:
import os
import pickle
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import xgboost as xgb
import optuna
from nilearn import plotting
from nilearn.image import load_img
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from nilearn import image, datasets, input_data
from nilearn.image import load_img, resample_img, resample_to_img
from nilearn.datasets import load_mni152_template
from xgboost import XGBClassifier
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\nilearn\input_data\__init__.py:23: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


# 1. Data

In [2]:
def collect_data(data_dir):
    """
    Traverse the data directory and return three lists:
      - anatomical image file paths
      - resting state fMRI file paths
      - labels (1 for ASD, 0 for TC)
    """
    fmri_files = []
    labels = []
    
    # subdirectories for each institution
    files = ['ABIDEII-EMC_1', 'ABIDEII-GU_1', 'ABIDEII-KKI_1',
             'ABIDEII-U_MIA_1', 'ABIDEII-NYU_2', 'ABIDEII-SU_2']
    i = 0
    
    for dataset in os.listdir(data_dir):
        # Paths for ASD and TC for both anatomical and fMRI data
        asd_path_fmri = os.path.join(data_dir, dataset, files[i], "ASD")
        tc_path_fmri = os.path.join(data_dir, dataset, files[i], "TC")
        
        # Process ASD subjects
        if os.path.exists(asd_path_fmri):
            for subject in os.listdir(asd_path_fmri):
                fmri_path = os.path.join(asd_path_fmri, subject, "session_1", "rest_1")
                if os.path.exists(fmri_path):
                    fmri_file = os.path.join(fmri_path, os.listdir(fmri_path)[0])
                    fmri_files.append(fmri_file)
                    labels.append(1)
                    
        # Process TC subjects
        if dataset != 'NYU Langone':
            if os.path.exists(tc_path_fmri):
                for subject in os.listdir(tc_path_fmri):
                    fmri_path = os.path.join(tc_path_fmri, subject, "session_1", "rest_1")
                    if os.path.exists(fmri_path):
                        fmri_file = os.path.join(fmri_path, os.listdir(fmri_path)[0])
                        fmri_files.append(fmri_file)
                        labels.append(0)
                        
        i += 1
        
    return fmri_files, labels

# 2. Dataset

In [3]:
class AutismDataset(Dataset):
    def __init__(self, fmri_files, labels, cache_file=None):
        """
        Loads fMRI ROI features using a masker.
        """
        self.fmri_files = fmri_files
        self.labels = labels

        # FMRI masker using the Harvard–Oxford Cortical Atlas
        atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
        self.masker_fmri = input_data.NiftiLabelsMasker(
            labels_img=atlas.maps,
            standardize=True,
            detrend=True,
            low_pass=0.1,
            high_pass=0.01,
            t_r=2.0,
            resampling_target="data"
        )
        
        # Load cached features if available
        if cache_file is not None and os.path.exists(cache_file):
            print(f"Loading cached fMRI features from {cache_file}...")
            with open(cache_file, 'rb') as f:
                self.fmri_features = pickle.load(f)
        else:
            print("Precomputing fMRI features...")
            self.fmri_features = []
            for fmri_path in tqdm(self.fmri_files, desc="Processing fMRI data"):
                fmri_img = load_img(fmri_path)
                time_series = self.masker_fmri.fit_transform(fmri_img)
                mean_ts = np.mean(time_series, axis=0)  # Compute the mean time series for each region.
                self.fmri_features.append(mean_ts.astype(np.float32))
            if cache_file is not None:
                with open(cache_file, 'wb') as f:
                    pickle.dump(self.fmri_features, f)
                print(f"Cached fMRI features saved to {cache_file}")

    def __len__(self):
        return len(self.fmri_files)

    def __getitem__(self, idx):
        # FMRI Features Processing
        expected_dim = 48
        fmri_feat = self.fmri_features[idx]
        if fmri_feat.shape[0] < expected_dim:
            fmri_feat = np.pad(fmri_feat, (0, expected_dim - fmri_feat.shape[0]), mode='constant')
        elif fmri_feat.shape[0] > expected_dim:
            fmri_feat = fmri_feat[:expected_dim]
        fmri_tensor = torch.tensor(fmri_feat)

        label = self.labels[idx]
        label_tensor = torch.tensor(label, dtype=torch.long)

        return fmri_tensor, label_tensor

In [4]:
data_dir = "Data"  # Data path
fmri_files, labels = collect_data(data_dir)
print(f"Found {len(fmri_files)} subjects.")

Found 468 subjects.


In [5]:
dataset = AutismDataset(fmri_files, labels, cache_file="files")

Loading cached fMRI features from files...


In [6]:
X_fmri, y_fmri = zip(*[dataset[i] for i in range(len(dataset))])
X_fmri = list(X_fmri)
y_fmri = list(y_fmri)

In [7]:
X_fmri = [x.numpy() for x in X_fmri]
y_fmri = [y.item() for y in y_fmri]

In [8]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fmri, y_fmri, test_size=0.2, random_state=42)

# 3. Training

In [9]:
# Set grid of parameters to explore
def objective(trial):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [50, 100, 150]),
        'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'gamma': trial.suggest_categorical('gamma', [0.0, 0.1, 0.2]),
        'random_state': 42,
        'use_label_encoder': False,
        'eval_metric': 'logloss',
    }
    
    model = XGBClassifier(**params)
    scores = cross_val_score(model, X_train, y_train,
                             cv=5, scoring='accuracy', n_jobs=-1)
    return scores.mean()

In [10]:
# Create a study object and specify the direction ('maximize' for accuracy)
study = optuna.create_study(direction='maximize')

# Start the optimization
study.optimize(objective, n_trials=300)

[I 2025-05-09 18:29:52,586] A new study created in memory with name: no-name-7d30c483-72db-4589-a3ea-a088e222409d
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:29:54,045] Trial 0 finished wit

[I 2025-05-09 18:29:58,688] Trial 7 finished with value: 0.6069189189189189 and parameters: {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.0069670939836865365, 'subsample': 0.7168009931644093, 'colsample_bytree': 0.7014625325166851, 'gamma': 0.0}. Best is trial 2 with value: 0.6177297297297297.
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/o

C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:30:00,899] Trial 15 finished with value: 0.590918918918919 and parameters: {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.00856299089521

[I 2025-05-09 18:30:03,484] Trial 22 finished with value: 0.5936576576576577 and parameters: {'n_estimators': 150, 'max_depth': 5, 'learning_rate': 0.007391177577428448, 'subsample': 0.848117841310414, 'colsample_bytree': 0.877262832528294, 'gamma': 0.0}. Best is trial 2 with value: 0.6177297297297297.
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/opt

C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:30:06,468] Trial 30 finished with value: 0.5935855855855856 and parameters: {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.0080384051451

...

C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:31:20,208] Trial 263 finished with value: 0.6176936936936936 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.01277513059

[I 2025-05-09 18:31:22,277] Trial 270 finished with value: 0.6256576576576577 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.013215536126258424, 'subsample': 0.8785775854689517, 'colsample_bytree': 0.7321158053957604, 'gamma': 0.2}. Best is trial 256 with value: 0.6284684684684685.
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optun

C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:31:24,507] Trial 278 finished with value: 0.6096936936936936 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.01270975329

[I 2025-05-09 18:31:26,671] Trial 285 finished with value: 0.6043243243243244 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.013506079868994063, 'subsample': 0.70726549767655, 'colsample_bytree': 0.7224089638527743, 'gamma': 0.2}. Best is trial 256 with value: 0.6284684684684685.
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/

C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 5, 7, 10),
C:\Users\eugen\anaconda3\envs\tensorflow\lib\site-packages\optuna\distributions.py:700: UserWarning: The distribution is specified by [5, 7] and step=10, but the range is not divisible by `step`. It will be replaced by [5, 5].
  warnings.warn(
C:\Users\eugen\AppData\Local\Temp\ipykernel_48140\3862269126.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.02),
[I 2025-05-09 18:31:29,137] Trial 293 finished with value: 0.6042522522522523 and parameters: {'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.01127721349

In [11]:
# Get the best hyperparameters
best_params = study.best_params

print("Best Hyperparameters:")
for key, value in best_params.items():
    print(f"{key}: {value}")

Best Hyperparameters:
n_estimators: 100
max_depth: 5
learning_rate: 0.012271823944489921
subsample: 0.8781993816751603
colsample_bytree: 0.7319464171000242
gamma: 0.2


In [12]:
# Rebuild the model with the best params
best_model = XGBClassifier(
    **best_params,               
    objective='binary:logistic', 
    eval_metric='error',         
    random_state=42,
    use_label_encoder=False 
)

# Fit on the full training set
best_model.fit(X_train, y_train, verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7319464171000242, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='error', feature_types=None, gamma=0.2,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.012271823944489921,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [13]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Set Accuracy: {accuracy:.2f}')

# Classification report
print('Classification Report:')
print(classification_report(y_test, y_pred))

Test Set Accuracy: 0.60
Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.90      0.70        50
           1       0.69      0.25      0.37        44

    accuracy                           0.60        94
   macro avg       0.63      0.57      0.53        94
weighted avg       0.63      0.60      0.55        94

